#### Generate ideas based on a similarity search stored in a vectordb

- idea is we grab the vectordb stored locally. if none, exists just create an empty one.
- provide that list of ideas i guess? maybe to the ollama prompt and tell it like hey(in a rlly good prompt) that you want to generate a script for instagram reel on a new topic that i ahve not gone over yet from this vectordb store.
- prvoide script

In [1]:
from langchain_community.llms import Ollama # will be used for prompting
from langchain.vectorstores import Chroma # will be used for vectordb store
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document # will be used to store text in vector store 
import uuid
from datetime import datetime
import os

Use the same model for embedding text (to compare idea similarity). Instantiate the local Ollama model (LLaMA3 in this case)

In [2]:
llm = Ollama(model="llama3")
embedding = OllamaEmbeddings(model="llama3")

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_57538/2493284979.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [3]:
PERSIST_DIR = "vectordb"

Loading vector DB
- supposed to be a function that loads the vector_db
- if it dne, create one
- create an instance of a vector db using chroma with that directory created(or used prev) and the embedding attached too

In [4]:
if not os.path.exists(PERSIST_DIR):
    os.makedirs(PERSIST_DIR)

db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_57538/1878142402.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)


This function will return a list of all prev saved ideas from the DB
- it'll run a similarity search on the db with the key "" --> itll return all documents (ideas)
- we will just limit to a 100 for now 
- also, this should be its own function that returns an array, but for testing purposes, ill just append to an array

In [5]:
past_ideas = []
docs = db.similarity_search("", k=100)
for doc in docs:
    past_ideas.append(doc.page_content)

Saving a new idea
- i am going to create a function for this now thatll be used later in the query function i am about to run 
- simple idea. create a document object with the idea as the page_content
- then add the document to the db object init earlier
- dont forget the persist!


UPDATE - adding meta data such as a unique id, timestamp (going to be used for getting the most recent), processed (so we can query and not have a large output)
- we just literally do the same thing but have a new parameter for metdata 

In [6]:
def save_new_idea(db, idea_text):
    # Create metadata
    unique_id = str(uuid.uuid4())
    timestamp = datetime.utcnow().isoformat()

    metadata = {
        "id": unique_id,
        "timestamp": timestamp,
        "processed": False
    }

    # Create Document object with metadata
    doc = Document(page_content=idea_text, metadata=metadata)

    # Add to DB and persist
    db.add_documents([doc])
    db.persist()

    print(f"Idea saved with ID: {unique_id}")

Generate new idea function:
- this function will take in past_ideas as a parameter. and it join them as a string with a new line character per idea. if no ideas in the past_ideas array, itll set joined_ideas as None yet.
- the prompt will be well strucutred as an f string thatll hold the joined_ideas var in it
- the llm will be invoked with teh prompt and expect a response

In [ ]:
def generate_new_idea(past_ideas):
    # Combine all past ideas into a single string, or show "None yet" if empty
    joined_ideas = "\n".join(past_ideas) if past_ideas else "None yet."

    # Craft a well-structured prompt asking for a creative, new idea
    prompt = f"""
    You are a creative AND technical Instagram assistant. Below is a list of past Instagram video topics:

    {joined_ideas}

    Come up with a completely new and original 50-second Instagram video script idea related to AWS services that hasn’t been covered yet.

    Your response must include only the raw spoken script — do NOT include a title, do NOT say anything like “Here’s a script,” and do NOT add any labels, bullet points, or section names. Do not explain what you’re doing. Do not add any extra commentary or setup.

    Just start the script as if you're speaking to the camera.

    The script should:
    - Begin with an attention-grabbing hook
    - Explain a specific AWS service in casual, plain English
    - Mention when or why someone would use it
    - Describe in one simple sentence how a developer might set it up or use it
    - Optionally include a surprising fact or tip
    - End with a call to action to engage the viewer

    Speak like you're talking to a tech-savvy friend. Avoid repeating any past topics listed above. Dive right into the script. Nothing else.
    """





    # Send prompt to the LLaMA model and get a generated response
    return llm.invoke(prompt)

Since we already kinda init the db (and loaded old prev data too if there even existed in the first place), im not going to include it in the function below. however, ill still include the generate_new_idea function since this is actually calling the llm and will also call the save_new_idea function

In [8]:
def run_idea_generation(db):
    new_idea = generate_new_idea(past_ideas)  # Generate a new idea
    save_new_idea(db, new_idea)           # Save the new idea to the DB
    return new_idea

In [9]:
print(run_idea_generation(db))

Idea saved with ID: f63e7113-3bca-4248-a484-aea11e882e21
Hey everyone, have you ever wondered how e-commerce sites can handle millions of orders and transactions seamlessly? Well, I'm here to tell you that it's all thanks to Amazon Simple Queue Service, or SQS for short.

So, what is SQS? Think of it like a message board where different services or microservices can leave notes for each other. When a customer places an order, the website sends a message to SQS, which then forwards it to the relevant teams, like fulfillment and shipping.

SQS is especially useful when you have multiple teams working on different parts of the same process. For instance, if your company uses AI-powered chatbots to handle customer inquiries, and another team handles order processing, SQS allows them to communicate with each other without getting bogged down in complex integrations.

To set up SQS, developers simply create a queue, define its properties, and then use APIs or SDKs to send and receive message

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_57538/327579714.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
